In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import os
os.chdir('../')

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import random
from src.dataset import *
from src.concept_vectors import *
from src.util import *
from src.plots import *
from src.hierarchy import *
from src.metrics import *
from src.models import * 
from src.create_vectors import *

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image, ImageOps
import pickle
import sklearn
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from scipy.spatial.distance import cdist


In [5]:
dataset = CUB_Dataset()

In [16]:
embedding_method = load_shapley_vectors_simple

NameError: name 'load_shplaey_vectors_simple' is not defined

In [33]:
for embedding_method in [load_label_vectors_simple,load_vae_concept_vectors_simple,load_cem_vectors_simple,load_concept2vec_vectors_simple,load_cem_stratified_vectors_simple]:
    embedding_method(dataset.get_attributes()[0],dataset,"",seed=43).shape

In [28]:
create_shapley_vectors(dataset.get_attributes(),dataset,"",seed=43)

Found 4796 validated image filenames belonging to 200 classes.
150/150 [==============================] - 790s 5s/step


In [35]:
for embedding_method in [load_shapley_vectors_simple]:
    print(embedding_method)
    t = truthfulness_metric_shapley(embedding_method,dataset,dataset.get_attributes(),[43,44,45],model_name="VGG16")
    print("{} {}".format(embedding_method,t))

<function load_shapley_vectors_simple at 0x7f7e5256f290>
Found 1198 validated image filenames belonging to 200 classes.
38/38 [==============================] - 201s 5s/step


FileNotFoundError: [Errno 2] No such file or directory: 'results/shapley/cub/44/has_bill_length::shorter_than_head.npy'

In [ ]:
create_shapley_vectors()

In [10]:
data = dataset.get_data()

In [7]:
img_paths = ['dataset/'+i['img_path'] for i in data]
labels = [str(i['class_label']) for i in data]

In [8]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

2023-03-17 14:17:09.669205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2023-03-17 14:17:09.669259: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-17 14:17:09.669293: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nemo.cl.cam.ac.uk): /proc/driver/nvidia/version does not exist
2023-03-17 14:17:09.669572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
for layer in vgg.layers:
    layer.trainable = False

In [10]:
fc1 = Flatten()(vgg.output)
fc1 = Dense(1024, activation='relu')(fc1)
output = Dense(200, activation='softmax')(fc1)

In [11]:
model = Model(inputs=vgg.input, outputs=output)

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])


/auto/homes/njr61/environments/mambaforge/envs/concepts/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
model.load_weights("results/models/vgg16_bottleneck.h5")

In [14]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32
image_size = (224, 224)
train_df = pd.DataFrame(zip(img_paths,labels), columns=["image_path", "label"])

In [15]:
data_valid = dataset.get_data(train=False)
img_paths_valid = ['dataset/'+i['img_path'] for i in data_valid]
labels_valid = [str(i['class_label']) for i in data_valid]
valid_df = pd.DataFrame(zip(img_paths_valid,labels_valid), columns=["image_path", "label"])

In [16]:
train_generator = datagen.flow_from_dataframe(dataframe=train_df,
                                              x_col="image_path",
                                              y_col="label",
                                              target_size=image_size,
                                              batch_size=batch_size,
                                              class_mode="categorical",
                                              shuffle=True)

Found 4796 validated image filenames belonging to 200 classes.


In [17]:
valid_generator = datagen.flow_from_dataframe(dataframe=valid_df,
                                              x_col="image_path",
                                              y_col="label",
                                              target_size=image_size,
                                              batch_size=batch_size,
                                              class_mode="categorical",
                                              shuffle=True)

Found 1198 validated image filenames belonging to 200 classes.


In [18]:
# model.fit(train_generator,
#           steps_per_epoch=len(train_generator),
#           epochs=10)

In [19]:
prediction = model.predict(valid_generator)

38/38 [==============================] - 195s 5s/step


In [20]:
prediction.shape

(1198, 200)

In [21]:
concepts = np.array([i['attribute_label'] for i in data_valid])

In [22]:
concepts.shape

(1198, 112)

In [23]:
[len(concepts[concepts[:,i] == 1]) for i in range(112)]

[102,
 74,
 482,
 303,
 293,
 305,
 55,
 470,
 247,
 161,
 276,
 363,
 82,
 484,
 205,
 155,
 79,
 110,
 155,
 195,
 526,
 180,
 757,
 100,
 99,
 266,
 329,
 48,
 308,
 139,
 139,
 368,
 212,
 231,
 345,
 138,
 109,
 61,
 245,
 82,
 102,
 160,
 203,
 443,
 146,
 63,
 118,
 242,
 437,
 78,
 1156,
 408,
 745,
 61,
 180,
 146,
 82,
 407,
 101,
 175,
 191,
 472,
 183,
 103,
 169,
 207,
 69,
 256,
 208,
 104,
 76,
 98,
 160,
 155,
 490,
 139,
 523,
 191,
 772,
 213,
 122,
 57,
 719,
 590,
 171,
 83,
 498,
 82,
 170,
 876,
 245,
 276,
 137,
 317,
 246,
 112,
 262,
 327,
 127,
 161,
 572,
 73,
 61,
 189,
 147,
 65,
 383,
 89,
 325,
 58,
 245,
 269]

In [24]:
def contribution_score(concept_num,prediction_num):
    positive_concept_indices = concepts[:,concept_num] == 1
    negative_concept_indices = concepts[:,concept_num] == 0
    
    logit_values = prediction[positive_concept_indices][:,prediction_num]
    logit_values_without = prediction[negative_concept_indices][:,prediction_num]
        
    return np.mean(logit_values)-np.mean(logit_values_without)

In [25]:
contribution_score(0,0)

0.0076353317

In [27]:
contribution_array = [[contribution_score(i,j) for j in range(200)] for i in range(112)]

In [28]:
contribution_array = np.array(contribution_array)

In [33]:
def crossentropy(u, v):
    ce = np.sum(-u*np.log(v) - (1-u)*np.log(1-v))
    dist = np.sqrt(ce)
    return dist


In [36]:
dist_array = cdist(contribution_array, contribution_array, metric='cosine')

In [41]:
similarity_array = 1-dist_array

In [43]:
largest_indices = np.argsort(similarity_array)[0, -5:]
largest_indices

array([ 14,  51, 100,  68,   0])